# Finding the best model

In [ ]:
def best_model(name,number_days_predict):
    Future_Price_ARIMA, mape_ARIMA, y_pred_ARIMA = model_ARIMA(name, number_days_predict)
    Future_Price_SARIMA, mape_SARIMA, y_pred_SARIMA = model_SARIMA(name, number_days_predict)
    #Future_Price_GARCH, mape_GARCH, y_pred_GARCH = model_GARCH('ACB', 7)
    Future_Price_ETS, mape_ETS, y_pred_ETS = model_ETS(name, number_days_predict)
    Future_Price_TBATS, mape_TBATS, y_pred_TBATS = model_TBATS(name, number_days_predict)
    Future_Price_Prophet, mape_Prophet, y_pred_Prophet = model_Prophet(name, number_days_predict)
    Future_Price_LSTM, mape_LSTM, y_pred_LSTM = model_LSTM(name, number_days_predict)
    weights, mape_ensemable = model_ensemble(name, 7, y_pred_ARIMA, y_pred_SARIMA,
                                    y_pred_ETS, y_pred_TBATS,
                                    y_pred_Prophet, y_pred_LSTM)
    Future_Price_ensemable = (
                                weights[0] * Future_Price_ARIMA + 
                                weights[1] * Future_Price_SARIMA + 
                                weights[2] * Future_Price_ETS + 
                                weights[3] * Future_Price_TBATS + 
                                weights[4] * Future_Price_Prophet + 
                                weights[5] * Future_Price_LSTM
                            )

    mape = np.array([mape_ARIMA,
                   mape_SARIMA,
                   mape_ETS,
                   mape_TBATS,
                   mape_Prophet,
                   mape_LSTM,
                    mape_ensemable])
    
    Future_Price = np.array([Future_Price_ARIMA,
                   Future_Price_SARIMA,
                   Future_Price_ETS,
                   Future_Price_TBATS,
                   Future_Price_Prophet,
                   Future_Price_LSTM,
                    Future_Price_ensemable])
    
    # Find index of minimum MAPE and return corresponding prediction
    best_index = np.argmin(mape)
    return mape[best_index], Future_Price[best_index]

# Create Database

In [ ]:
def predict_stocks_database(stock_symbols, number_days_predict, all_stock_data):
    mape = []
    Future_Price = []
    Profit = []

    for name in stock_symbols:
        # Lọc dữ liệu theo mã cổ phiếu
        data = all_stock_data[all_stock_data['ticker'] == name]
        
        # Gọi hàm best_model
        mape_name, Future_Price_name = best_model(name, number_days_predict)
        
        # Tính lợi nhuận dự đoán
        last_close_price = data['close'].iloc[-1]  # Lấy giá đóng cửa cuối cùng
        profit = Future_Price_name / last_close_price - 1

        # Lưu kết quả
        mape.append(mape_name)
        Future_Price.append(Future_Price_name)
        Profit.append(profit)

    # Tạo DataFrame kết quả
    df = pd.DataFrame({
        'Stock': stock_symbols[:len(mape)],  # Chỉ lấy số lượng phù hợp
        'mape': mape,
        'Future_Price': Future_Price,
        'Profit': Profit
    })

    return df


# Testing

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit


In [ ]:
# Lấy data để phục vụ test model 
%run Collect_Data_Function.ipynb
# Lấy danh sách các mã cổ phiếu trên VN30
stock_symbols = ["PHR","MBB"]  # Cột chứa mã cổ phiếu
all_stock_data = data_collect(stock_symbols)
all_stock_data.head()

In [ ]:
df = predict_stocks_database(stock_symbols, 5, all_stock_data)
print(df)